<a href="https://colab.research.google.com/github/Maruf346/AI-ML-with-python/blob/main/Transfer_Learning_with_Pretrained_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Use transfer learning with pretrained models like ResNet or VGG.**


Tasks:

1. Load a pretrained model using torchvision.models.


2. Replace the final layer with a new classifier (e.g., 2 classes).


3. Freeze earlier layers, train only the classifier.


4. Evaluate performance.

In [ ]:
# =======================================
# STEP 0: Install & Import Libraries
# =======================================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, Dataset
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =======================================
# STEP 1: Data Preparation
# =======================================
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # ResNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Download CIFAR-10
train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# Filter cats (3) and dogs (5)
def filter_dataset(dataset, classes_to_keep=[3, 5]):
    indices = [i for i, (_, label) in enumerate(dataset) if label in classes_to_keep]
    return Subset(dataset, indices)

train_data_filtered = filter_dataset(train_data)
test_data_filtered = filter_dataset(test_data)

# Wrap Subset to relabel cats=0, dogs=1
class RelabelledDataset(Dataset):
    def __init__(self, subset):
        self.subset = subset
    def __getitem__(self, idx):
        x, y = self.subset[idx]
        new_label = 0 if y == 3 else 1
        return x, new_label
    def __len__(self):
        return len(self.subset)

train_data = RelabelledDataset(train_data_filtered)
test_data = RelabelledDataset(test_data_filtered)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# =======================================
# STEP 2: Load Pretrained ResNet18
# =======================================
model = models.resnet18(pretrained=True)

# Freeze earlier layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 classes: cat, dog
model = model.to(device)

# =======================================
# STEP 3: Loss & Optimizer
# =======================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# =======================================
# STEP 4: Training Loop
# =======================================
epochs = 3
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # ✅ no need to wrap with torch.tensor

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")

# =======================================
# STEP 5: Evaluation
# =======================================
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")
